In [1]:
import snntorch as snn
from snntorch import spikeplot as splt
from snntorch import spikegen

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt

from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint
import pytorch_lightning as L

from pytorch_lightning import Trainer
import wandb

import os

from RSNN import SpikeSynth

In [2]:
# Our data in in the shape: trainings samples(28k) * number of time steps (100 + 6) * time dimension(1)
# The time steps is voltage over time
data = torch.load(f'./data/dataset.ds')

print(data.keys())
print(data['X_train'].shape)
print(data['Y_train'].shape)

dict_keys(['X_train', 'Y_train', 'X_valid', 'Y_valid', 'X_test', 'Y_test'])
torch.Size([28797, 106, 1])
torch.Size([28797, 100])


In [3]:
# Extract tensors
X_train, Y_train = data['X_train'], data['Y_train']
X_valid, Y_valid = data['X_valid'], data['Y_valid']
X_test, Y_test = data['X_test'], data['Y_test']

train_dataset = TensorDataset(X_train, Y_train)
valid_dataset = TensorDataset(X_valid, Y_valid)
test_dataset  = TensorDataset(X_test, Y_test)

In [4]:
 model = SpikeSynth(
        optimizer_class=torch.optim.AdamW,
        beta=0.9,
        lr=0.01,
        num_hidden=10,
        batch_size=1024,
        gamma=0.9,
        num_hidden_layers=2
    )

In [8]:
trainer = Trainer(
    #fast_dev_run=True,
    max_epochs=5,
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
)

# Train the model
trainer.fit(model)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name         | Type       | Params | Mode 
----------------------------------------------------
0 | lif_layers   | ModuleList | 410    | train
1 | output_layer | Linear     | 11     | train
----------------------------------------------------
221       Trainable params
200       Non-trainable params
421       Total params
0.002     Total estimated model params size (MB)
6         Modules in train mode
0         Modules in eval mode


Sanity Checking: |             | 0/? [00:00<?, ?it/s]

/nix/store/qsg98l1y41k4530xsn0ss82xrd03095c-python3-3.12.11-env/lib/python3.12/site-packages/torch/nn/modules/loss.py:616: UserWarning: Using a target size (torch.Size([576, 100])) that is different to the input size (torch.Size([576, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/nix/store/qsg98l1y41k4530xsn0ss82xrd03095c-python3-3.12.11-env/lib/python3.12/site-packages/pytorch_lightning/loops/fit_loop.py:310: The number of training batches (29) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |                    | 0/? [00:00<?, ?it/s]

/nix/store/qsg98l1y41k4530xsn0ss82xrd03095c-python3-3.12.11-env/lib/python3.12/site-packages/torch/nn/modules/loss.py:616: UserWarning: Using a target size (torch.Size([125, 100])) that is different to the input size (torch.Size([125, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


...

Validation: |                  | 0/? [00:00<?, ?it/s]

Validation: |                  | 0/? [00:00<?, ?it/s]

Validation: |                  | 0/? [00:00<?, ?it/s]

Validation: |                  | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


In [6]:
x = torch.rand(128, 300, 1)
y = model(x)
y.size()

torch.Size([128, 294])